## Summary

This code is the runnable python notebook version of `bio_tagging_email.py`

This file is used to BIO tagging on the email information. One synthetic dataset will be created.

The output file will be in the format of spaCy Doc file.

special tricks applied: <br>
None


special dependencies: <br>
1. The `extracted_emails.json` generated by `clean_tags.py`.

## Import

In [ ]:
import collections
import json
import os
import random
from ast import literal_eval
from pathlib import Path
from random import randrange

import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from spacy.tokens import Doc, DocBin
from spacy.util import filter_spans
from tqdm import tqdm

## Helper Functions

In [ ]:
#helper function
def add_email(example, email_dict):
    """
    add the email information to the example as a separate column.
    
    Args:
      example: the row in the cicero df
      email_dict: the dictionary that stores the email information
    
    Returns:
      the row with the email information added
      
    """
    example['extracted_email'] = email_dict.get(str(example['id']))
    return example

In [ ]:
#helper function
def choice_excluding(lst, exception_index, target_number):
    """
    random choose a list of items from a list, 
    excluding the item at the exception_index
    
    Args: 
      lst: lst for randomly choose emails from
      exception_index: the index of the item that should be excluded
      target_number: the number of items that should be chosen

    Returns:
        a list of emails from the lst
    
    """
    refined_list = lst[:exception_index] + lst[exception_index + 1:]
    refined_list = [
        literal_eval(n) for n in refined_list if not (isinstance(n, float))
    ]
    refined_list = [n for n in refined_list if n]
    return random.choices(refined_list, k=target_number)

## Load and Prepare Data

In [ ]:
# your own path to cicero data
CICERO_DATA_PATH = ''

# your own path to extracted email file
EXTRACTED_EMAIL_PATH = ''

In [ ]:
# load the extracted email
with open("EXTRACTED_EMAIL_PATH", "r") as f:
      email_dict = json.load(f)

In [ ]:
# load the Cicero data
cicero_df = pd.read_csv('CICERO_DATA_PATH',\
                  error_bad_lines=False)

In [ ]:
# add the email information to the cicero_df
cicero_df = cicero_df.apply(lambda x: add_email(x, email_dict), axis=1)

In [ ]:
# get the list of emails
email_list = cicero_df['extracted_email'].tolist()
email_list = [str(n) for n in email_list]

## Bio Tagging

In [ ]:
output = Path("./email/")

In [ ]:
# detect if the output directory exists
if not os.path.exists(output):
    os.makedirs(output)

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
#random selecting and bio_tagging
inserted_data = []
for n in tqdm(range(len(cicero_df))):
    attribute_list = list(cicero_df.iloc[n].dropna())
    attribute_dict = dict(cicero_df.iloc[n].dropna())

    # randomly choose 3 list of emails for the email_list
    # and then concatenate them together
    # the first 5 emails in the concatenated list will be the email_list
    # these emails will be treated as NEGATIVE cases
    attribute_list = sum(choice_excluding(email_list, n, 3), [])[:5]

    # get the correct email from the cicero_df
    # these emails will be treated as POSITIVE cases
    if 'email_1' in attribute_dict:
        attribute_list += [attribute_dict.get('email_1')]
    if 'email_2' in attribute_dict:
        attribute_list += [attribute_dict.get('email_2')]

    # shuffle the list of emails
    random.shuffle(attribute_list)

    # add the name of the politician at the beginning of the list
    # since the position of name is fixed, the language model will learn it as the hint
    # to identify the email
    attribute_list = [attribute_dict.get('first_name')
                      ] + [attribute_dict.get('last_name')] + attribute_list
    ruler = nlp.add_pipe("span_ruler")
    bio_tag_pattern_list = []

    if 'email_1' in attribute_dict:
        bio_tag_pattern_list.append({
            "label": 'EMAIL',
            'pattern': attribute_dict['email_1']
        })
    if 'email_2' in attribute_dict:
        bio_tag_pattern_list.append({
            "label": 'EMAIL',
            'pattern': attribute_dict['email_2']
        })

    ruler.add_patterns(bio_tag_pattern_list)
    input_text = ' '.join(attribute_list)
    doc = nlp(input_text)
    doc.ents = filter_spans(doc.spans["ruler"])
    dataset = []
    dataset.append(doc)
    inserted_data.append(doc)
    # remove the ruler and initialize a new one for each politician/data point
    nlp.remove_pipe("span_ruler")

  0%|          | 4/1919 [00:00<01:55, 16.55it/s]/usr/local/lib/python3.8/dist-packages/spacy/pipeline/span_ruler.py:507: UserWarning: [W036] The component 'span_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
  1%|          | 10/1919 [00:00<02:16, 14.00it/s]/usr/local/lib/python3.8/dist-packages/spacy/pipeline/span_ruler.py:507: UserWarning: [W036] The component 'span_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
  2%|▏         | 30/1919 [00:02<02:12, 14.30it/s]/usr/local/lib/python3.8/dist-packages/spacy/pipeline/span_ruler.py:507: UserWarning: [W036] The component 'span_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
  2%|▏         | 36/1919 [00:02<02:06, 14.86it/s]/usr/local/lib/python3.8/dist-packages/spacy/pipeline/span_ruler.py:507: UserWarning: [W036] The component 'span_ruler' does not have any patterns defined.
  warnings.warn(War

In [ ]:
# remove the empty data point
inserted_data = [doc for doc in inserted_data if len(doc) > 0]

In [ ]:
# split the tagged data into training set and test set
train, dev = train_test_split(inserted_data, test_size = 0.1, random_state=42)

In [ ]:
train_db = DocBin()
for n in train:
    train_db.add(n)
train_db.to_disk(output / "train.spacy")

In [ ]:
dev_db = DocBin()
for n in dev:
    dev_db.add(n)
dev_db.to_disk(output / "dev.spacy")